In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import csv
import gmaps

# Google API Key
from config import gkey

# Configure gmaps with API key
gmaps.configure(api_key=gkey)

%matplotlib inline

In [2]:
# Import CSV
g_lookup_pd = pd.read_csv("Resources/Data Filtration.csv")

In [3]:
# Rename columns for readability
g_lookup_pd_rn = g_lookup_pd.rename(columns={"restaurant_name": "Restaurant Name",
                                             "inspection_type": "Inspection Type",
                                             "inspection_date": "Inspection Date",
                                             "inspection_score": "Inspection Score",
                                             "street_number": "Street Number",
                                             "street_name": "Street Name",
                                             "street_unit": "Street Unit",
                                             "zip_code": "Zip",
                                             "street_address": "Street Address",
                                             "inspection_month": "Inspection Month",
                                             "inspection_year": "Inspection Year",
                                                })

g_lookup_pd_rn["Lat"] = ""
g_lookup_pd_rn["Long"] = ""
g_lookup_pd_rn["Type"] = ""
g_lookup_pd_rn["Rating"] = ""
g_lookup_pd_rn["Rating Count"] = ""


g_lookup_pd_rn

,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,Inspection Year,Lat,Long,Type,Rating,Rating Count
0,"""LET'S DO LUNCH""",Routine,8/28/2019,100,1720,REGAL,#105,75235,1720 REGAL ROW #105,8/1/2019,FY2019,,,,,
1,$ STUFF & MORE,Routine,12/28/2018,93,4702,EAST GRAND,NaN,75223,4702 EAST GRAND AVE,12/1/2018,FY2019,,,,,
2,A & A MART,Routine,5/14/2019,81,309,MARSALIS,NaN,75203,309 S MARSALIS AVE,5/1/2019,FY2019,,,,,
3,A & J BAKERY,Routine,4/30/2019,92,3515,OAK LAWN,NaN,75219,3515 OAK LAWN AVE,4/1/2019,FY2019,,,,,
4,A & K MART,Routine,2/7/2019,98,5723,SINGLETON,NaN,75212,5723 SINGLETON BLVD,2/1/2019,FY2019,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5102,ZOE'S KITCHEN,Routine,10/21/2019,90,8018,PARK,#100,75231,8018 PARK LN #100,10/1/2019,FY2020,,,,,
5103,ZOE'S KITCHEN,Routine,9/23/2019,89,6025,ROYAL,104,75230,6025 ROYAL LN STE 104,9/1/2019,FY2019,,,,,
5104,ZOE'S KITCHEN,Routine,8/14/2019,94,5710,LOVERS,#108,75209,5710 W LOVERS LN #108,8/1/2019,FY2019,,,,,
5105,ZRG CONCEPTS,Routine,9/16/2019,96,2505,FITZHUGH,NaN,75204,2505 N FITZHUGH AVE,9/1/2019,FY2019,,,,,


In [4]:
# Create a copy of re-organized Data
g_lookup_pd_filter = g_lookup_pd_rn.copy()

# Filter by Inspection Score
#g_lookup_pd_filter = g_lookup_pd_filter[g_lookup_pd_filter['Inspection Score'] == 83]

# Filter by Zip
#g_lookup_pd_filter = g_lookup_pd_filter[g_lookup_pd_filter['Zip'] == 75251]

#g_lookup_pd_filter = g_lookup_pd_filter.reset_index()

#g_lookup_pd_filter

,index,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,Inspection Year,Lat,Long,Type,Rating,Rating Count
0,235,BANNER PLACE SANDWICH SHOP,Routine,12/7/2018,87,12770,COIT,#160,75251,12770 COIT RD #160,12/1/2018,FY2019,,,,,
1,281,BENIHANA,Routine,10/22/2019,83,7775,BANNER,NaN,75251,7775 BANNER DR,10/1/2019,FY2020,,,,,
2,646,CENTRAL CAFE,Routine,5/18/2018,90,12712,PARK CENTRAL,#B100,75251,12712 PARK CENTRAL DR #B100,5/1/2018,FY2018,,,,,
3,898,CORNER BAKERY CAFE,Routine,1/9/2020,93,5225,BELTLINE,NaN,75251,5225 BELTLINE RD 258,1/1/2020,FY2020,,,,,
4,1449,EVA CAFE,Routine,9/17/2019,80,12404,PARK CENTRAL,#110,75251,12404 PARK CENTRAL DR #110,9/1/2019,FY2019,,,,,
5,1539,FLO'S CAFE,Routine,9/6/2019,86,12271,COIT,NaN,75251,12271 COIT RD,9/1/2019,FY2019,,,,,
6,1677,GATEWAY NEWSTAND,Routine,5/18/2018,92,12770,MERIT,#123,75251,12770 MERIT DR #123,5/1/2018,FY2018,,,,,
7,1797,GREEN OAKS HOSPITAL DIETARY,Routine,12/12/2019,93,7808,CLODUS FIELDS,NaN,75251,7808 CLODUS FIELDS DR,12/1/2019,FY2020,,,,,
8,2098,JACK IN THE BOX,Routine,7/5/2019,90,7940,LBJ,NaN,75251,7940 LBJ FRWY,7/1/2019,FY2019,,,,,
9,2655,MAASAI GRILL,Follow-up,2/6/2019,81,8059,LBJ,NaN,75251,8059 LBJ FRWY,2/1/2019,FY2019,,,,,


In [5]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the g_lookup_pd_org and run a lat/long search for each city
for index, row in g_lookup_pd_filter.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/geocode/json"

        street = row['Street Address']
        zip_code = row['Zip']
        restaurantname = row['Restaurant Name']
        restaurantname = restaurantname.title()

        # update address key value
        params['address'] = f"{street},{zip_code}"

        # make request
        cities_lat_lng = requests.get(base_url, params=params)

        # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
        
        # convert to json
        cities_lat_lng = cities_lat_lng.json()
        
        g_lookup_pd_filter.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        #lat = g_lookup_pd_filter.loc[index, "Lat"]
        
        g_lookup_pd_filter.loc[index, "Long"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
        #long = g_lookup_pd_filter.loc[index, "Long"]
        
        print(f'''[OK!] {restaurantname} found!''')            

    except:
        print(f'''[ERROR!] {restaurantname} missing field/result.''')

        
        pass

[OK!] Banner Place Sandwich Shop found!
[OK!] Benihana found!
[OK!] Central Cafe found!
[OK!] Corner Bakery Cafe found!
[OK!] Eva Cafe found!
[OK!] Flo'S Cafe found!
[OK!] Gateway Newstand found!
[OK!] Green Oaks Hospital Dietary found!
[OK!] Jack In The Box found!
[OK!] Maasai Grill found!
[OK!] Merit Tower Cafe found!
[OK!] Murphy'S Deli found!
[OK!] R & R Cafe found!
[OK!] R & R Cafe found!
[OK!] Residence Inn Park Centrl found!
[OK!] Rio Bravo Mexican Restaurant Seafood & Bar found!
[OK!] Starbucks found!
[OK!] Sundry found!
[OK!] Sushi Axiom found!
[OK!] T Bone Pickens found!
[OK!] Taco Casa found!
[OK!] Taste Date Cafe found!
[OK!] Westin (Banquet Kitchen) found!
[OK!] Winston Sundry Shop found!
[OK!] Yiddishe Kup found!


In [6]:
# Check Data
g_lookup_pd_filter

,index,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,Inspection Year,Lat,Long,Type,Rating,Rating Count
0,235,BANNER PLACE SANDWICH SHOP,Routine,12/7/2018,87,12770,COIT,#160,75251,12770 COIT RD #160,12/1/2018,FY2019,32.9216,-96.7683,,,
1,281,BENIHANA,Routine,10/22/2019,83,7775,BANNER,NaN,75251,7775 BANNER DR,10/1/2019,FY2020,32.9216,-96.7729,,,
2,646,CENTRAL CAFE,Routine,5/18/2018,90,12712,PARK CENTRAL,#B100,75251,12712 PARK CENTRAL DR #B100,5/1/2018,FY2018,32.9213,-96.7763,,,
3,898,CORNER BAKERY CAFE,Routine,1/9/2020,93,5225,BELTLINE,NaN,75251,5225 BELTLINE RD 258,1/1/2020,FY2020,32.9547,-96.8174,,,
4,1449,EVA CAFE,Routine,9/17/2019,80,12404,PARK CENTRAL,#110,75251,12404 PARK CENTRAL DR #110,9/1/2019,FY2019,32.9172,-96.7753,,,
5,1539,FLO'S CAFE,Routine,9/6/2019,86,12271,COIT,NaN,75251,12271 COIT RD,9/1/2019,FY2019,32.9164,-96.7703,,,
6,1677,GATEWAY NEWSTAND,Routine,5/18/2018,92,12770,MERIT,#123,75251,12770 MERIT DR #123,5/1/2018,FY2018,32.9231,-96.7714,,,
7,1797,GREEN OAKS HOSPITAL DIETARY,Routine,12/12/2019,93,7808,CLODUS FIELDS,NaN,75251,7808 CLODUS FIELDS DR,12/1/2019,FY2020,32.9131,-96.772,,,
8,2098,JACK IN THE BOX,Routine,7/5/2019,90,7940,LBJ,NaN,75251,7940 LBJ FRWY,7/1/2019,FY2019,32.9231,-96.7696,,,
9,2655,MAASAI GRILL,Follow-up,2/6/2019,81,8059,LBJ,NaN,75251,8059 LBJ FRWY,2/1/2019,FY2019,32.9253,-96.7676,,,


In [7]:
# params dictionary to update each iteration
# radius of 50 has been giving me a 90% match for Type, Rating, and Rating Count
params = {
    "radius": 50,
    "types": "restaurant",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in g_lookup_pd_filter.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Long"]
    restaurantname = row["Restaurant Name"]
    restaurantname = restaurantname.title()

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        g_lookup_pd_filter.loc[index, "Type"] = name_address["results"][0]["types"][0].title()
        g_lookup_pd_filter.loc[index, "Rating"] = name_address["results"][0]["rating"]
        g_lookup_pd_filter.loc[index, "Rating Count"] = name_address["results"][0]["user_ratings_total"]
        
        print(f'''[OK!] {restaurantname} found!''')
        
    
    except (KeyError, IndexError):
        print(f'''[ERROR!] {restaurantname} missing field/result.''')

[OK!] Banner Place Sandwich Shop found!
[OK!] Benihana found!
[OK!] Central Cafe found!
[OK!] Corner Bakery Cafe found!
[OK!] Eva Cafe found!
[ERROR!] Flo'S Cafe missing field/result.
[ERROR!] Gateway Newstand missing field/result.
[ERROR!] Green Oaks Hospital Dietary missing field/result.
[OK!] Jack In The Box found!
[ERROR!] Maasai Grill missing field/result.
[ERROR!] Merit Tower Cafe missing field/result.
[ERROR!] Murphy'S Deli missing field/result.
[ERROR!] R & R Cafe missing field/result.
[ERROR!] R & R Cafe missing field/result.
[ERROR!] Residence Inn Park Centrl missing field/result.
[ERROR!] Rio Bravo Mexican Restaurant Seafood & Bar missing field/result.
[ERROR!] Starbucks missing field/result.
[ERROR!] Sundry missing field/result.
[OK!] Sushi Axiom found!
[ERROR!] T Bone Pickens missing field/result.
[OK!] Taco Casa found!
[ERROR!] Taste Date Cafe missing field/result.
[OK!] Westin (Banquet Kitchen) found!
[ERROR!] Winston Sundry Shop missing field/result.
[ERROR!] Yiddishe K

In [8]:
g_lookup_pd_filter

,index,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,Inspection Year,Lat,Long,Type,Rating,Rating Count
0,235,BANNER PLACE SANDWICH SHOP,Routine,12/7/2018,87,12770,COIT,#160,75251,12770 COIT RD #160,12/1/2018,FY2019,32.9216,-96.7683,Meal_Takeaway,5,4
1,281,BENIHANA,Routine,10/22/2019,83,7775,BANNER,NaN,75251,7775 BANNER DR,10/1/2019,FY2020,32.9216,-96.7729,Restaurant,4.2,2025
2,646,CENTRAL CAFE,Routine,5/18/2018,90,12712,PARK CENTRAL,#B100,75251,12712 PARK CENTRAL DR #B100,5/1/2018,FY2018,32.9213,-96.7763,Bakery,4.1,28
3,898,CORNER BAKERY CAFE,Routine,1/9/2020,93,5225,BELTLINE,NaN,75251,5225 BELTLINE RD 258,1/1/2020,FY2020,32.9547,-96.8174,Bakery,4.2,451
4,1449,EVA CAFE,Routine,9/17/2019,80,12404,PARK CENTRAL,#110,75251,12404 PARK CENTRAL DR #110,9/1/2019,FY2019,32.9172,-96.7753,Restaurant,4.5,2
5,1539,FLO'S CAFE,Routine,9/6/2019,86,12271,COIT,NaN,75251,12271 COIT RD,9/1/2019,FY2019,32.9164,-96.7703,,,
6,1677,GATEWAY NEWSTAND,Routine,5/18/2018,92,12770,MERIT,#123,75251,12770 MERIT DR #123,5/1/2018,FY2018,32.9231,-96.7714,,,
7,1797,GREEN OAKS HOSPITAL DIETARY,Routine,12/12/2019,93,7808,CLODUS FIELDS,NaN,75251,7808 CLODUS FIELDS DR,12/1/2019,FY2020,32.9131,-96.772,,,
8,2098,JACK IN THE BOX,Routine,7/5/2019,90,7940,LBJ,NaN,75251,7940 LBJ FRWY,7/1/2019,FY2019,32.9231,-96.7696,Restaurant,3.9,954
9,2655,MAASAI GRILL,Follow-up,2/6/2019,81,8059,LBJ,NaN,75251,8059 LBJ FRWY,2/1/2019,FY2019,32.9253,-96.7676,,,


In [9]:
# Save as CSV
g_lookup_pd_filter.to_csv("Google Lat Long Results.csv")